# Using Jamba Instruct on Bedrock for Standard Operating Procedures and Regulatory Document Analysis
In the pharmaceutical industry, regulatory compliance is crucial for ensuring the safety and efficacy of drug products. Regulatory bodies like the FDA establish detailed guidelines for drug manufacturing processes, which companies must align with through their Standard Operating Procedures (SOPs). Drug compliance officers play a key role in this by reviewing SOPs to identify and rectify any inconsistencies with regulatory standards. Automation tools, such as AI-based comparison systems, can streamline this process by efficiently analyzing and highlighting discrepancies between SOPs and regulatory documents, thereby enhancing compliance accuracy and ensuring the continuous quality of pharmaceutical products.

In this notebook, we use AI21's Jamba Instruct model analyze Subpart E Title 21 of the [Code of Fedral Regulations](https://www.govinfo.gov/app/collection/cfr/2023/title21/chapterI/subchapterC/part211), this section is nearly 30 pages long. We will use Jamba-Instruct to perform contradiction detection, to see if a mock Standard Operating Procedure is consistent with the the regulatory document.

In [3]:
# Required imports
import os
import json
import requests
import time
import boto3
# Function to read the contents of the files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def _call_bedrock_jamba(prompt,**kwargs):
    bedrock = boto3.client(service_name="bedrock-runtime")
    body={
        "messages":[{"role":"user","content":prompt}],
        "max_tokens": 1024,
        "top_p": 0.8,
        "temperature": 0.7,
    }
    body.update(kwargs)
    body = json.dumps(body)
    
    modelId = "ai21.jamba-instruct-v1:0"
    
    accept = "application/json"
    contentType = "application/json"
    
    response = bedrock.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    result=json.loads(response.get('body').read())
    return result['choices'][0]['message']['content']

def call_bedrock_jamba(prompt, **kwargs):
    attempts = 0
    while attempts < 5:
        try:
            return _call_bedrock_jamba(prompt, **kwargs)
        except Exception as e:
            print(e)
            attempts += 1
            print(f"Attempt {attempts}: Failed to call API, retrying in 3 seconds...")
            time.sleep(3)
    raise Exception("Failed to complete the API call after 5 attempts")

Read in the SOP and Regulatory Document

In [4]:
# Paths to the text files
sample_sop_path = 'sample_sop.txt'
subpart_path = 'Subpart_E_163_190.txt'

# Read the contents of the files
sample_sop_content = read_file(sample_sop_path)
subpart_content = read_file(subpart_path)


## Prompt Jamba-Instruct model

Next, we will prompt the Jamba-Instruct model to look for specific contradictions. Note that we also are using a system prompt, to have Jamba Instruct take on the role of a compliance officer.

In [5]:
# Create a prompt for the AI21 endpoint
comparison_prompt = f"""
You are a drug compliance officer. You examine Standard Operating Procedures (SOP) to see if they are consistent with regulations.


Explain clearly what the contradiction is, what the SOP says, and how it contradicts the Regulatory Document.

Standard Operating Procedure: 
{sample_sop_content}

Regulatory Document:
{subpart_content}

Please analyze the documents documents and determine if the content of Standard Operating Procedure is consistent with the Regulatory Document. 
Highlight any discrepancies. 

Specifically, focus on any **blatant contradictions**, where "Standard Operating Procedure" specifies a procedure that directly contradicts the "Regulatory" Document.


Contradictions Result:
"""

#print(comparison_prompt)
# Call the API
comparison_result = call_bedrock_jamba(comparison_prompt,temperature=0)




Inspect the results

In [6]:
# Print the comparison result
print("Comparison Result:")
print(comparison_result)

Comparison Result:
 The provided documents contain extensive Standard Operating Procedures (SOPs) for receiving, storing, sampling, testing, and handling of paracetamol, and a detailed Regulatory Document outlining the requirements for components, drug product containers, and closures under 21 CFR parts 211 and 212. Upon review, several discrepancies and contradictions were identified between the SOP and the Regulatory Document. Here are the key findings:

1. **Receipt of Paracetamol Shipment (SOP vs. Regulatory Document)**


  * **SOP**: Specifies that paracetamol can be immediately released to storage and is not required to be quarantined prior to testing.
  * **Regulatory Document**: Requires that each lot of components, drug product containers, and closures shall be withheld from use until the lot has been sampled, tested, or examined, as appropriate, and released for use by the quality control unit (21 CFR 211.84(a)).
	
2. **Storage of Paracetamol (SOP vs. Regulatory Document)**



## Conclusion
As you can see from the results above, Jamba Instruct correctly picks up on the fact that in the SOP, Paracetamol is relesed to storage and not quarantined prior to being tested. This flatly contradicts the Regulatory document that requires that the drug product be quarantied prior to testing.  

(note that you may see different results yourself, depending on the temperature you use when calling Jamba)